# Deep Learning for Time Series Forecasting

This notebook demonstrates using LSTM neural networks for time series forecasting.

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.models.lstm_model import LSTMForecaster
from src.utils.helpers import generate_sample_data, create_sequences
from src.preprocessing.data_loader import train_test_split_ts
from src.preprocessing.transformers import Normalizer
from src.evaluation.metrics import calculate_metrics, print_metrics
from src.visualization.plots import plot_forecast

%matplotlib inline

## 1. Prepare Data

In [ ]:
# Generate sample data
data = generate_sample_data(n_samples=500, trend=True, seasonality=True)

# Normalize data
normalizer = Normalizer(method='minmax')
data_normalized = normalizer.fit_transform(data.values)

# Split into train and test
split_idx = int(len(data_normalized) * 0.8)
train_data = data_normalized[:split_idx]
test_data = data_normalized[split_idx:]

print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

## 2. Create Sequences for LSTM

In [ ]:
# Create sequences
n_steps = 30
X_train, y_train = create_sequences(train_data, n_steps=n_steps)
X_test, y_test = create_sequences(test_data, n_steps=n_steps)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

## 3. Build and Train LSTM Model

In [ ]:
# Create LSTM model
lstm_model = LSTMForecaster(
    n_steps=n_steps,
    n_features=1,
    units=50,
    dropout=0.2
)

# Train model
lstm_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

## 4. Make Predictions

In [ ]:
# Make predictions
predictions_normalized = lstm_model.predict(X_test)

# Inverse transform to original scale
predictions = normalizer.inverse_transform(predictions_normalized)
y_test_original = normalizer.inverse_transform(y_test.reshape(-1, 1))

print(f"Predictions shape: {predictions.shape}")

## 5. Evaluate Model

In [ ]:
# Calculate metrics
metrics = calculate_metrics(y_test_original.flatten(), predictions.flatten())
print("LSTM Model Performance:")
print_metrics(metrics)

## 6. Visualize Results

In [ ]:
# Create visualization
plt.figure(figsize=(14, 6))
plt.plot(y_test_original, label='Actual', linewidth=2, marker='o', markersize=3)
plt.plot(predictions, label='Predicted', linewidth=2, linestyle='--', marker='x', markersize=3)
plt.title('LSTM Forecast vs Actual', fontsize=14, fontweight='bold')
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Training History

In [ ]:
# Plot training history
history = lstm_model.history

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated:
- Preparing data for LSTM models
- Building and training LSTM networks
- Making forecasts with trained models
- Evaluating model performance